## 第三章

In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('/tmp/data/',one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
mnist.train.images.shape

(55000, 784)

In [5]:
mnist.train.labels.shape

(55000, 10)

In [7]:
X=tf.placeholder(tf.float32,shape=[None,784],name='X')
Y=tf.placeholder(tf.float32,shape=[None,10],name='Y')

W1=tf.Variable(tf.random_normal([784,256]))
b1=tf.Variable(tf.random_normal([256]))
W2=tf.Variable(tf.random_normal([256,256]))
b2=tf.Variable(tf.random_normal([256]))
W3=tf.Variable(tf.random_normal([256,10]))
b3=tf.Variable(tf.random_normal([10]))

layer_1=tf.nn.relu(tf.add(tf.matmul(X,W1),b1))
layer_2=tf.nn.relu(tf.add(tf.matmul(layer_1,W2),b2))
output=tf.add(tf.matmul(layer_2,W3),b3)

cross_entropy=tf.nn.softmax_cross_entropy_with_logits(logits=output,labels=Y)
cost=tf.reduce_mean(cross_entropy)

train_step=tf.train.AdamOptimizer(0.001).minimize(cost)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(10):
        avg_cost=0
        total_batch=int(mnist.train.num_examples/128)
        for j in range(total_batch):
            batch_x,batch_y=mnist.train.next_batch(128)
            _,loss=sess.run([train_step,cost],feed_dict={X:batch_x,Y:batch_y})
            avg_cost+=loss/total_batch
        print("Echos:"+str(i)+"loss:"+str(avg_cost))
    correct_prediction=tf.equal(tf.argmax(output,1),tf.argmax(Y,1))
    #计算准确率
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    print("Accuracy:",accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))
        
        

Echos:0loss:194.424714131
Echos:1loss:46.4549056078
Echos:2loss:29.4935318551
Echos:3loss:20.8782569373
Echos:4loss:15.3581682157
Echos:5loss:11.6873083688
Echos:6loss:9.05309097827
Echos:7loss:6.98041409854
Echos:8loss:5.35094055061
Echos:9loss:4.23372574576
Accuracy: 0.9337


## 第五章

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/tmp/data/",one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
import numpy as np
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1))
def bias_variable(shape):
    return tf.Variable(tf.constant(0.1,shape=shape))
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
def max_pool(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

X=tf.placeholder(tf.float32,shape=[None,28,28,1],name='X')
Y=tf.placeholder(tf.float32,shape=[None,10],name='Y')

def qian(X):
    w_conv1=weight_variable([5,5,1,32])
    b_conv1=bias_variable([32])
    h_conv1=tf.nn.relu(conv2d(X,w_conv1)+b_conv1)
    h_pool1=max_pool(h_conv1)
    
    w_conv2=weight_variable([5,5,32,64])
    b_conv2=bias_variable([64])
    h_conv2=tf.nn.relu(conv2d(h_pool1,w_conv2)+b_conv2)
    h_pool2=max_pool(h_conv2)
    
    w_fc1=weight_variable([7*7*64,64])
    b_fc1=bias_variable([64])
    h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64])
    h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,w_fc1)+b_fc1)
    
    #keep_prob=tf.placeholder(tf.float32,shape=[1])
    #h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)
    
    w_fc2=weight_variable([64,10])
    b_fc2=bias_variable([10])
    
    pred=tf.nn.softmax(tf.matmul(h_fc1,w_fc2)+b_fc2)
    
    return pred

pred=qian(X)

cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=Y))
train_step=tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

correct_prediction=tf.equal(tf.argmax(pred,1),tf.argmax(Y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    tf.global_variables_initializer().run() 
    for i in range(2):
        avg_total=0
        num_total=int(mnist.train.num_examples/128)
        for j in range(num_total):
            batch_x,batch_y=mnist.train.next_batch(128)
            batch=np.reshape(batch_x,(128,28,28,1))
            _,loss=sess.run([train_step,cross_entropy],feed_dict={X:batch,Y:batch_y})
            avg_total+=loss/128
        if(i%2==0):
            print("echo:"+str(i)+"loss:"+str(avg_total))
    test_input=mnist.test.images
    test_input=np.reshape(test_input,[test_input.shape[0],28,28,1])
    print("test accuracy %g" % accuracy.eval(feed_dict={X:test_input,Y:mnist.test.labels}))
    

echo:0loss:5.7629293371
test accuracy 0.8834


## 第六章

In [2]:
from datetime import datetime
import math
import time
import tensorflow as tf
batch_size=32
num_batchs=100
def print_activations(t):
    print(t.op.name," ",t.get_shape().as_list)

    
def inference(images):
    parameters=[]
    with tf.name_scope('conv1') as scope:
        kernel=tf.Variable(tf.truncated_normal([11,11,3,64],stddev=0.1),name="weights")
        conv=tf.nn.conv2d(images,kernel,[1,4,4,1],padding='SAME')
        biases=tf.Variable(tf.constant(0.0,shape=[64],dtype=tf.float32),trainable=True,name='biases')
        bias=tf.nn.bias_add(conv,biases)
        conv1=tf.nn.relu(bias,name=scope)
        
        print_activations(conv1)
        parameters+=[kernel,biases]
        pool1=tf.nn.max_pool(conv1,ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME',name='pool1')
        print_activations(pool1)
    with tf.name_scope('conv2') as scope:
        kernel=tf.Variable(tf.truncated_normal([5,5,64,192],stddev=0.1,name="weights"))
        conv=tf.nn.conv2d(pool1,kernel,strides=[1,1,1,1],padding='SAME')
        biases=tf.Variable(tf.constant(0.0,shape=[192],dtype=tf.float32),trainable=True,name='biases')
        bias=tf.nn.bias_add(conv,biases)
        conv2=tf.nn.relu(bias,name=scope)
        parameters+=[kernel,biases]
        print_activations(conv2)
        pool2=tf.nn.max_pool(conv2,ksize=[1,3,3,1],strides=[1,2,2,1],padding='SMAE',name='pool2')
        print_activations(pool2)
    with tf.name_scope('conv3') as scope:
        kernel=tf.Variable(tf.truncated_normal([3,3,192,384],stddev=0.1,name='weights'))
        conv=tf.nn.conv2d(pool2,kernel,strides=[1,1,1,1],padding='SAME')
        biases=tf.Variable(tf.constant(0.0,shape=[384],dtype=tf.float32),trainable=True,name='biases')
        bias=tf.nn.bias_add(conv,biases)
        conv3=tf.nn.relu(bias,name=scope)
        parameters+=[kernel,biases]
        print_activations(conv3)
    with tf.name_scope('conv4') as scope:
        kernel=tf.Variable(tf.truncated_normal([3,3,384,256],stddev=0.1,name='weights'))
        conv=tf.nn.conv2d(conv3,kernel,strides=[1,1,1,1],padding='SAME')
        biases=tf.Variable(tf.constant(0.0,shape=[256],dtype=tf.float32),trainable=True,name='biases')
        bias=tf.nn.bias_add(conv,biases)
        conv4=tf.nn.relu(bias,name=scope)
        parameters+=[kernel,biases]
        print_activations(conv4)
        
    with tf.name_scope('conv5') as scope:
        kernel=tf.Variable(tf.truncated_normal([3,3,256,256],stddev=0.1,name='weights'))
        conv=tf.nn.conv2d(conv4,kernel,strides=[1,1,1,1],padding='SAME')
        biases=tf.Variable(tf.constant(0.0,shape=[256],dtype=tf.float32),trainable=True,name='biases')
        bias=tf.nn.bias_add(conv,biases)
        conv5=tf.nn.relu(bias,name=scope)
        parameters+=[kernel,biases]
        print_activations(conv5)
        pool5=tf.nn.max_pool(conv5,ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME',name='pool5')
        print(pool5)
    return pool5,parameters
